# Week 11 - Explainable AI

### Aims

This week will focus on tools available to explain black box ML models, specifically looking at some of the tools available in `sklearn` and in `shap`:

>* Partial Dependence Plots
>* Individual Conditional Expectations
>* Permutation Importance 
>* SHAP values

Note: this week's material does not need to be submitted and the material is NOT examinable, but it could be useful for your project, depending on your choice of models.

# Setup 

Let's load the packages needed for this workshop (more will be loaded through the workshop).

In [ ]:
# Data libraries
import pandas as pd
import numpy as np

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn modules
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score

# Example 1: Dependent vs Independent Features

## Partial Dependence Plots and Individual Conditional Expectation Plots

To better understand partial dependence plots (PDPs) and individual conditional expectation (ICE) plots, we will start by simulating a data set. Notice that `x1` and `x2` are nonlinearly dependent and independent of `x3`, while the target `y` is an additive nonlinear function of `x1` and `x3` only with additive errors.

In [ ]:
# Generate data
np.random.RandomState(seed=11205)
N = 1000 # sample size

# Generate features (x1 and x2 are dependent but independent of x3)
x1 = np.random.uniform(-2,2,size=(N,1))
x2 = (x1**3)/4 +np.random.normal(0,0.1,(N,1))
x3 = np.random.uniform(-2,2,size=(N,1))
X = np.hstack([x1,x2,x3])

# Generate output (only depends on x1 and x3)
y = np.sin(x1) + np.exp(x3)/8 + np.random.normal(0,0.1,(N,1))
y = y.reshape(-1,)

# Pair plot
d = pd.DataFrame(X, columns=['x1','x2','x3'])
d['y'] = y
sns.pairplot(d)
plt.show()

Let's start by fitting a model. We will fit a random forests regression model.

In [ ]:
# Import RF
from sklearn.ensemble import RandomForestRegressor

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle= True, test_size = 0.2, random_state=0)

# Fit the model
rf = RandomForestRegressor(n_estimators=500, max_features='sqrt', random_state=0)
rf.fit(X_train,y_train)

# Print metrics
yhat = rf.predict(X_test)
print('RMSE:', root_mean_squared_error(yhat,y_test))
print('R2:', r2_score(yhat,y_test))

We see that the R squared value is quite high. Now let's use PDPs to investigate the effect of each feature. Remember that PDPs aim to estimate and visualize the marginal effects of each feature. The marginal effects are defined as:
$$ f_d(x_d) = \int f(x_1,\ldots,x_D) p_{-d|d}(x_1,\ldots,x_{d-1}, x_{d+1},\ldots,x_D \mid x_d) dx_1\ldots dx_{d-1} dx_{d+1}\ldots dx_D,$$
where $f(\mathbf{x})$ represents the regression function and $p_{-d|d}(\cdot| x_d)$ represents the conditional density of all features except the $d\text{th}$ feature given the $d\text{th}$ feature. Note that if the features are independent, then:
$$ p_{-d|d}(x_1,\ldots,x_{d-1}, x_{d+1},\ldots,x_D \mid x_d) =  p_{-d}(x_1,\ldots,x_{d-1}, x_{d+1},\ldots,x_D ),$$
where $p_{-d}(\cdot)$ represents the marginal density of all features except the $d\text{th}$ feature. 

In **Partial Dependence Plots**, we assume independence of the features and use a Monte Carlo approximation of this integral (i.e. estimate the marginal by the empirical distribution), by computing the average across the training data:
$$ \widehat{f}(x_d) = \frac{1}{N} \sum_{n=1}^N f(x_{n,1},\ldots,x_{n,d-1},x_d,x_{n,d+1},\ldots,x_D).$$

In this example, since we generated the data, we know the know the true marginal effects, which are:
$$ f_1(x_1)= \sin(x_1)+0+ \int_{-2}^2 \frac{\exp(x_3)}{8} \frac{1}{4} d x_3 = \sin(x_1) +\frac{\exp(2)-\exp(-2)}{32}.$$
$$ f_2(x_2) = \int_{-2}^2 \sin(x_1) \frac{1}{4} d x_1 + 0 + \int_{-2}^2 \frac{\exp(x_3)}{8} \frac{1}{4}d x_3 = \frac{\exp(2)-\exp(-2)}{32}.$$
$$ f_3(x_3) = \int_{-2}^2 \sin(x_1) \frac{1}{4} d x_1 + 0  + \frac{\exp(x_3)}{8} = \frac{\exp(x_3)}{8}.$$
In the plots below, we compare the estimated marginal effects with the truth.

We will compute the PDP using the [`PartialDependenceDisplay.from_estimator`](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.PartialDependenceDisplay.html#sklearn.inspection.PartialDependenceDisplay.from_estimator). Some important options to specify include:
- `estimator`: the fitted model
- `X`: the training feature matrix
- `features`: a list of the features for which to create PDPs; this can contain integers (e.g. `[0,3,5]`) or pairs of integers (e.g.`[0,1,[0,1]]`) for pairwise PDPs.  
- `kind`: if equal to `'average'` computes the PDPs.

Note: at the bottom of the PDP, the bars help to visualize the distribution of the feature in order to understand regions where data may be limited.

For further details and examples, see [PDPs](https://scikit-learn.org/stable/auto_examples/inspection/plot_partial_dependence.html).

In [ ]:
from sklearn.inspection import PartialDependenceDisplay

fig, ax = plt.subplots(ncols=3, nrows=1, figsize=(15, 5), constrained_layout=True)
PartialDependenceDisplay.from_estimator(
    rf,
    X_train,
    features=[0,1,2],
    kind='average',
    ax=ax
)
ax[0].plot(np.sort(x1,axis=0),np.sin(np.sort(x1,axis=0))+(np.exp(2)-np.exp(-2))/32,c ='r',label='True')
ax[0].set_ylim(-1,1.25)
ax[0].legend()
ax[1].plot(x2,np.zeros_like(x2)+ (np.exp(2)-np.exp(-2))/32,c ='r',label='True')
ax[1].legend()
ax[2].plot(np.sort(x3,axis=0),np.exp(np.sort(x3,axis=0))/8,c ='r',label='True')
ax[2].legend()
plt.show()

The main disadvantage of PDPs is that they assumes **indepedence** between the sets of features. Indeed, above we see that the marginal effect of `x3` is both well approximated in the PDP, as this feature was generated independently of the other two features. But the marginal effect is not well approximated in the PDP for the first two features. 

Let's consider a contrived example of how we might misinterpret the effects of the features. Suppose we are predicting medical charges `y` as a function of age `x1`, distance to a major city `x2`, and bmi `x3`, and that in the dataset collected, the younger individuals all lived closer to major cities. In this case, we would wrongly conclude that there is a sharp effect depending on whether individuals live close to cities or not, and this would effect our decision to insurance these individuals and the value to charge for insurance.

The PDPs are computed as an average of the regression curves across all data points, thus we can also plot the **Individual Conditional Expectation Plot** for any data point $\mathbf{x}_n$, which is defined as:
$$ \widehat{f}_n(x_d) = f(x_{n,1},\ldots,x_{n,d-1},x_d,x_{n,d+1},\ldots,x_D).$$

This is again computed using [`PartialDependenceDisplay.from_estimator`](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.PartialDependenceDisplay.html#sklearn.inspection.PartialDependenceDisplay.from_estimator) but for ICE, we set
- `kind`: equal to `'individual'` (or `'both'` to show both the PDP and ICE).
You may also want to set `subsample` to an integer (e.g. `subsample=50`) to reduce both computations and the number of lines on the plot.

In [ ]:
# ICE: use the option subsample to reduce the number of ICE curves drawn
fig, ax = plt.subplots(ncols=3, nrows=1, figsize=(15, 5), constrained_layout=True)
PartialDependenceDisplay.from_estimator(
    rf,
    X_train,
    features=[0,1,2],
    kind='both',
    # subsample = 50, 
    ax=ax
)
ax[0].plot(np.sort(x1,axis=0),np.sin(np.sort(x1,axis=0))+(np.exp(2)-np.exp(-2))/32,c ='r',label='True')
ax[0].legend()
ax[1].plot(x2,np.zeros_like(x2)+ (np.exp(2)-np.exp(-2))/32,c ='r',label='True')
ax[1].legend()
ax[2].plot(np.sort(x3,axis=0),np.exp(np.sort(x3,axis=0))/8,c ='r',label='True')
ax[2].legend()
plt.show()

In this case, the ICE curves are close to parallel (different levels of the other covariates, simply shift the curve up or down). This suggests that there are no interesting interactions between the features. Since we simulated the data under an additive assumption (no interactions), this can be expected.

## Permutation Importance

To get an understanding of the importance of the features, for random forests and bagged trees, we can compute the feature importance, which first, measures the reduction in the impurity (in the regression case, the RSS) due to splits associated to the feature for each tree in the ensemble, and then averages this across all trees.  However, this is specific to tree-based models.

The **permutation-based importance measures** are a general tool that can be used for any estimator to quantify the importance of the feature. 

Let's start by computing and visualizing the RF feature importances that we learned about in Week 8. As we learned, the correlation between `x1` and `x2` dilutes and inflates the importance of these two features, respectively (see plot below). In particular, from this, we would conclude that `x2` is more important than `x3`.

In [ ]:
# Compute the standard deviation 
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)

# Create a pandas series with the feature importances
importances_rf = pd.Series(
    rf.feature_importances_, index=['x1','x2','x3']
).sort_values(ascending=True)

# Plot the feature importances
ax = importances_rf.plot.barh(xerr=std)
ax.set_title("Random Forest: Feature Importances")
ax.figure.tight_layout()

Let's now compute the permutation importance. 

**Permutation feature importance** measures the increase in the prediction error of the model after we permuted the feature’s values, which breaks the relationship between the feature and the true outcome. The concept is straightforward: we measure the importance of a feature by calculating the increase in the model’s prediction error after permuting the feature. A feature is “important” if shuffling its values increases the model error, because in this case the model relied on the feature for the prediction. A feature is “unimportant” if shuffling its values leaves the model error unchanged, because in this case the model ignored the feature for the prediction. It can be computed on both the training and test data, with the latter the recommended option. 

In `sklearn`, it is implemented in [`permutation_importance`](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html). Some important options include:
- `scoring` which specifies how we measure the prediction error.
- `n_repeats` which is the number of times to permute a feature. This defaults to 5, but larger values will give more accurate estimates, although at additional computational cost.

The function returns a dictionary containing:
- `importances_mean`: the mean of the feature importance over all repetitions for each feature. 
- `importances_std`: the standard deviation of the feature importance over all repetitions for each feature.
- `importances`: a matrix containing the importance for each feature and each repetition. 

In [ ]:
from sklearn.inspection import permutation_importance

# Permutation Importance on the training data
pi_train = permutation_importance(
    rf, X_train, y_train, n_repeats=10, random_state=42
)

# Permutation Importance on the test data
pi_test = permutation_importance(
    rf, X_test, y_test, n_repeats=10, random_state=42
)

# Create a pandas series with the permutation importances for the training data
pi_train_rf = pd.Series(
    pi_train.importances_mean, index=['x1','x2','x3']
).sort_values(ascending=True)

# Create a pandas series with the permutation importances for the test data
pi_test_rf = pd.Series(
    pi_test.importances_mean, index=['x1','x2','x3']
).sort_values(ascending=True)

# Plot the feature importances
fig, ax = plt.subplots(1,2, figsize = (12,5))
pi_train_rf.plot.barh(xerr=pi_train.importances_std, ax = ax[0])
ax[0].set_title("Random Forest: Permutation Importances (Train)")
pi_test_rf.plot.barh(xerr=pi_test.importances_std, ax=ax[1])
ax[1].set_title("Random Forest: Permutation Importances (Test)")
plt.show()

Notice that the permutation-based importance still suffers from multicolinearity, but it is improved compared with the RF feature importance. Specifically, `x2` is no longer more _important_ than `x3`.

For further examples and details, see [permutation vs feature importance](https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance.html#sphx-glr-auto-examples-inspection-plot-permutation-importance-py) and [permutation and multicolinearity](https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance_multicollinear.html).

## SHAP values

Shapley values are a widely used tool to explain machine learning models and were developed from ideas in  cooperative game theory.

We won't go into the details of SHAP values (for details see the Chapters on [Shapley values](https://christophm.github.io/interpretable-ml-book/shapley.html) and [SHAP](https://christophm.github.io/interpretable-ml-book/shap.html) from the Interpretable ML book).  

SHAP is a local model-agnostic method, meaning that it explains individual predictions and can be used on any supervised model. For each point $\mathbf{x}_n$, the interpretation of the Shapley value for the $d\text{th}$ feature is: the value of the $d\text{th}$ feature contributed to the prediction $\widehat{f}(\mathbf{x}_n)$ of this particular data point compared to the average prediction $\mathbb{E}[\widehat{f}(\mathbf{x})]$ for the dataset (where the expectation is computed with respect to the joint density of the features and approximated by a Monte Carlo estimate using the empirical distribution).

Let's see an example to understand better. First, we estimate the shap values. 

In [ ]:
# Install shap if needed
#!pip install shap
import shap

# Subsample the training data to speed up computations
X100 = shap.utils.sample(X_train, 100)

# Compute the shap explainer and values
explainer = shap.Explainer(rf, X100, feature_names= ['x1','x2','x3'])
shap_values = explainer(X_train)

Let's pick an individual data point (here I have chosen an index of 20, try changing the index!). The first plot below is simply the PDP and ICE plot, but with the SHAP value for this feature illustrated, as the contribution of this feature to the prediction compared to the average.

This is further illustrated by the waterfall plot, which visualizes how much each of the features contribute to the prediction compared with the average.

In [ ]:
# Choose a data point
sample_ind = 20

# Plot the PDP for a specific feature and indicate the points shap value
shap.partial_dependence_plot(
    0,
    rf.predict,
    X,
    feature_names= ['x1','x2','x3'],
    model_expected_value=True,
    feature_expected_value=True,
    ice=True,
    shap_values=shap_values[sample_ind : sample_ind + 1, :]
)
# Show a waterfall plot to explain the decision
shap.plots.waterfall(shap_values[sample_ind], max_display=14)


For this data point, we observe that feature `x1` contributes the most, followed by `x2` and `x3` which have similar contributions but in opposite directions. But how much does this change across the data points?

To summarize across multiple data points, we can compute and visualize the mean absolute SHAP value of each feature over all data points.

In [ ]:
shap.plots.bar(shap_values)

Again, we observe that SHAP values are sensitive to multicollinearity in the features. And in this case, the mean SHAP values provide similar conclusions compared with the permutation measures (although the relative importance of `x1` is slightly higher for permutation importance). 

To gain a further understanding, the beeswarm plot displays SHAP values for each feature across all data points, with colors indicating how the SHAP values relate to the feature values.

In [ ]:
shap.plots.beeswarm(shap_values)

In general, we observe that for all features, lower values decrease the prediction compared with baseline, while higher values increase it. We can also appreciate the importance of `x1`, with many SHAP values around the highest contribution of $\pm 0.6$. 

For additional examples and illustrations of SHAP values, see [examples](https://shap.readthedocs.io/en/latest/example_notebooks/overviews/An%20introduction%20to%20explainable%20AI%20with%20Shapley%20values.html#)

# Example 2: Independent features and interactions 

We will modify the data generating mechanism, to i) remove the dependence between `x1` and `x2` and 2) assume an interaction between `x1` and `x3`.

In [ ]:
# Generate data
np.random.RandomState(seed=11205)
N = 1000 # sample size

# Generate features 
X = np.random.uniform(-2,2,size=(N,3))

# Generate output (only depends on x1 and x3)
y = np.sin(X[:,0])*np.exp(X[:,2])/8 + np.random.normal(0,0.1,(N,1)).reshape(-1,)

# Pair plot
d = pd.DataFrame(X, columns=['x1','x2','x3'])
d['y'] = y
sns.pairplot(d)
plt.show()

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle= True, test_size = 0.2, random_state=0)

### 🚩 Exercise 1

Fit a random forest regression model to the data and compute the RMSE and R2 on test data. How well does the model perform on the test data?

In [ ]:
# Fit the model


# Print metrics


### 🚩 Exercise 2

The true marginal effects in this example are:
$$ f_1(x_1)= \sin(x_1)\int_{-2}^2 \frac{\exp(x_3)}{8} \frac{1}{4} d x_3 = \sin(x_1)*\frac{\exp(2)-\exp(-2)}{32}.$$
$$ f_2(x_2) = \int_{-2}^2 \sin(x_1) \frac{1}{4} d x_1 * \int_{-2}^2 \frac{\exp(x_3)}{8} \frac{1}{4}d x_3 = 0.$$
$$ f_3(x_3) = \int_{-2}^2 \sin(x_1) \frac{1}{4} d x_1 * \frac{\exp(x_3)}{8} = 0.$$

Plot the PDPs for each feature and overlay the true marginal effects. How well are the true effects captured?

### 🚩 Exercise 3

Visualize the ICEs for each feature. Do the ICEs suggest something more interesting beyond the PDPs?

### 🚩 Exercise 4

Compute the both the RF feature importance and permutation importance (on both the test and train data). Which best reflects the importance of the features (based on your knowledge of the true data generating mechanism)?

### 🚩 Exercise 5

Compute and plot the SHAP values. How do the mean SHAP values reflect the importance of the features (based on your knowledge of the true data generating mechanism)? Use a beeswarm plot to visualize the individual SHAP values; does this provide any interesting insight beyond looking at the mean SHAP values?

### 🚩 Exercise 6

Lastly, let's change the target and problem into a classification tasks to understand how we can use PDPs to visualize the predicted probability and decision boundary. 

Consider the following data:
- Fit a classification model of your choice (e.g. `RandomForestClassifier`)
- Show the PDPs for each pair of features. (_Hint_: set `features=[[0,1],[0,2],[1,2]]`. You may also want to decrease the `grid_resolution` and set `subsample` to speed up the computation).
- Overlay the decision boundary on the PDP for the pair `x1` and `x3`.
- Plot the PDPs and ICEs for each feature individually.

In [ ]:
# Generate output (only depends on x1 and x3)
p = 1/(1 +np.exp(10*(np.sin(2*X[:,0])*(X[:,0]-2)+X[:,2])))
y = np.random.binomial(1, p)

# Pair plot
d['y'] = y
sns.pairplot(d, hue ='y')
plt.show()

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle= True, test_size = 0.2, random_state=0)